In [1]:
#Import dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from sqlalchemy import create_engine

In [2]:
#store link into variable
url = 'https://en.wikipedia.org/wiki/Wikipedia:10,000_most_common_passwords'
response = requests.get(url)

In [3]:
#Extract data via BS4
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
#Extract the lists of common passwords
all_lists = soup.find_all('div', class_="div-col columns column-width")

In [5]:
#Exract the top 100 list
top_100_list = []
top_100 = all_lists[0].find_all('li')
x=np.arange(len(top_100))
for x in np.arange(len(top_100)):
    top_100_list.append(top_100[x].text)

In [6]:
#Extract the remaining 9900 passwords
leftover_passwords_list = []
leftover_passwords = all_lists[1].find_all('li')
x=np.arange(len(leftover_passwords))
for x in np.arange(len(leftover_passwords)):
    leftover_passwords_list.append(leftover_passwords[x].text)

In [7]:
#Store lists into dataframe
top100_df = pd.DataFrame(top_100_list)
remainder_df = pd.DataFrame(leftover_passwords_list)

In [11]:
remainder_df = remainder_df.drop([0])

In [7]:
#Store lists into dataframe, rename column header, remove empty strings
top100_df = pd.DataFrame(top_100_list)
top100_df= top100_df.rename(columns={0:"common_passwords"})
remainder_df = pd.DataFrame(leftover_passwords_list)
remainder_df= remainder_df.rename(columns={0:"common_passwords"})
remainder_df = remainder_df.drop([0])

In [13]:
#Merge both dataframes into one
common_passwords_df = pd.merge(left=top100_df, right=remainder_df, 
                               on="common_passwords", how='outer')
common_passwords_df

,common_passwords
0,123456
1,password
2,12345678
3,qwerty
4,123456789
...,...
9995,caca
9996,c2h5oh
9997,bubbles1
9998,brook


In [9]:
#create connection to SQL DB
rds_connection_string = "postgres:<enter password here>@localhost:5432/security_breaches_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
#list table names in db
engine.table_names()

['common_passwords']

In [11]:
common_passwords_df.to_sql(name='common_passwords', con=engine, if_exists='append', index=False)